In [2]:
from device import Device
from rgf_time import GreensFunction

In [3]:
device = Device()
rgf = GreensFunction(device_state=device)

In [7]:
rgf.self_energy(0, .1)

starting SGF
Iteration 1: error=5.25e+01, inversion=21.960317s, multiplication=2.730384s, iter total=24.807945s
Iteration 2: error=5.69e+01, inversion=13.935818s, multiplication=2.013495s, iter total=16.000318s
Iteration 3: error=3.33e+01, inversion=10.393139s, multiplication=1.460699s, iter total=11.878843s
Iteration 4: error=4.07e+01, inversion=7.694952s, multiplication=0.217288s, iter total=7.955860s
Iteration 5: error=2.94e+01, inversion=0.754926s, multiplication=0.115719s, iter total=0.870821s
Iteration 6: error=7.36e+01, inversion=8.285090s, multiplication=1.730797s, iter total=10.055900s
Iteration 7: error=3.97e+01, inversion=5.788330s, multiplication=0.725705s, iter total=6.547690s
Iteration 8: error=2.18e+01, inversion=0.521894s, multiplication=0.030938s, iter total=0.552994s
Iteration 9: error=1.43e+01, inversion=0.035465s, multiplication=0.056570s, iter total=0.105263s
Iteration 10: error=8.79e+00, inversion=0.419508s, multiplication=0.033032s, iter total=0.452757s
Iteration

(array([[-3.41856394+0.5402295j ,  2.30182215-0.18586316j,
          2.66142562-0.56953796j, ...,  0.        +0.j        ,
          0.        +0.j        , -0.        +0.j        ],
        [ 2.18344571-0.4082672j , -2.10061808+0.11881057j,
         -1.44721751+0.43876454j, ...,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 2.68069194-0.3753421j , -1.59579764+0.09946287j,
         -2.45889654+0.39074252j, ...,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        ...,
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        , ...,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        , ...,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0. 

In [6]:
rgf.rgf(0, .1)

[rgf] Hamiltonian creation: 12.414561 s
starting SGF
Iteration 1: error=5.25e+01, inversion=8.834763s, multiplication=1.921363s, iter total=10.784160s
Iteration 2: error=5.69e+01, inversion=15.113208s, multiplication=3.018562s, iter total=18.197794s
Iteration 3: error=3.33e+01, inversion=13.644825s, multiplication=2.114341s, iter total=15.791180s
Iteration 4: error=4.07e+01, inversion=12.676421s, multiplication=1.688478s, iter total=14.391710s
Iteration 5: error=2.94e+01, inversion=10.407997s, multiplication=1.929769s, iter total=12.375996s
Iteration 6: error=7.36e+01, inversion=10.585873s, multiplication=1.614916s, iter total=12.229971s
Iteration 7: error=3.97e+01, inversion=10.058065s, multiplication=1.218510s, iter total=11.328607s
Iteration 8: error=2.18e+01, inversion=1.337137s, multiplication=0.110845s, iter total=1.448134s
Iteration 9: error=1.43e+01, inversion=8.051804s, multiplication=1.351506s, iter total=9.429352s
Iteration 10: error=8.79e+00, inversion=0.232295s, multiplica

(array([ 0.00557029+8.84057093e-03j,  0.03151129+1.81142583e-02j,
         0.03942832+1.69409916e-02j, ..., -0.05973207-4.44241897e-03j,
        -0.07093485-3.63195194e-03j, -0.05660872-2.67933980e-05j]),
 array([-2.05918139e-02+1.45042984e-03j, -2.02014592e-02+5.66781825e-03j,
        -1.85260165e-02+4.57549743e-03j, ...,
         2.01607131e-03-2.83256998e-04j,  2.19519469e-03+3.01719829e-04j,
         2.18962104e-05+3.39693787e-05j]),
 array([[-1.080459  +0.j        ,  0.59413036+0.11837644j,
          0.94488006-0.01926632j, ...,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.59413036-0.11837644j, -0.23762113+0.j        ,
         -0.53822741+0.14858013j, ...,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.94488006+0.01926632j, -0.53822741-0.14858013j,
         -0.78148503+0.j        , ...,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        .